**NOTE! Change run time type to T4 GPU inorder to speed up the process time.**

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tweets-blogs-news-swiftkey-dataset-4million:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F6261%2F9186%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240901%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240901T142205Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D36ffc720428430960a7d0b7a44b0ff2d6011bbb1e559da0fc3ecd1f29b69e4b0c729d6d7195a8ddf46b1db9ffc1a701f670926e9a417b2050b132bd52a6e6355f672ae04512185c27440464778334ddc11bee13e87b540505340bdebcea015471343d86bb6a15f3367ea2a866bffda3cb07879c12bbc9811260dae8c3974868bd24bb687f78c4b8d9c467a901e7450f36a5a44b7d1e474a36cc5b5e291b69cc754955317e472b5a145445892d25e5a83d62cc1d5b0b4b8cb6e9a97f062dd542accc102bde8dfc0336556dae263987cc797752e99f94fa32a7d9320d9f8155c3a0680c6d5310fe3b5f7fdd2ebc195def4ac74e80e6ff1f1389c76d50a7a605e84'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a id="basic-setup"></a>
# 📂 Basic Setup

In this **hidden** code cell, we'll import our packages. As we'll implement n-gram models from scratch we'll just use numpy and Additionally nltk  (just for Tokenization).

In [2]:
%%capture

## Importing Packages
import math
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Basic File Paths
data_dir = "../input/tweets-blogs-news-swiftkey-dataset-4million/final/en_US"
file_path = data_dir + "/en_US.twitter.txt"

## nltk settings
nltk.data.path.append(data_dir)
nltk.download('punkt')

## Opening the File in read mode ("r")
with open(file_path, "r") as f:
    data = f.read()

<a id="pre-process"></a>
# 🧽 Pre-Processing pipeline (Note: As our dataset is quite big, this process will take up to 10 min)

We create a simple pipeline function which:

* splits the datasets by the `\n` character

* remove leading and trailing spaces

* drop empty sentences.

* Tokenize sentences using `nltk.word_tokenize`

In [3]:
def preprocess_pipeline(data) -> 'list':

    # Split by newline character
    sentences = data.split('\n')

    # Remove leading and trailing spaces
    sentences = [s.strip() for s in sentences]

    # Drop Empty Sentences
    sentences = [s for s in sentences if len(s) > 0]

    # Empty List to hold Tokenized Sentences
    tokenized = []

    # Iterate through sentences
    for sentence in sentences:

        # Convert to lowercase
        sentence = sentence.lower()

        # Convert to a list of words
        token = nltk.word_tokenize(sentence)

        # Append to list
        tokenized.append(token)

    return tokenized


## Pass our data to this function
tokenized_sentences = preprocess_pipeline(data)

<a id="split"></a>
# ✂️ Splitting into Train, Valid and Test

In [4]:
## Obtain Train and Test Split
train, test = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)

## Obtain Train and Validation Split
train, val = train_test_split(train, test_size=0.25, random_state=42)

<a id="clean"></a>
# 🧹 Cleaning the Data

<a id="frequency"></a>
## 📔 Creating a Frequency Dictionary

As our dataset is quite big, we'll only use those words that appear `k` times in our dataset. In this function, we'll create a frequency dictionary for our vocabulary.

In [5]:
def count_the_words(sentences) -> 'dict':

  # Creating a Dictionary of counts
  word_counts = {}

  # Iterating over sentences
  for sentence in sentences:

    # Iterating over Tokens
    for token in sentence:

      # Add count for new word
      if token not in word_counts.keys():
        word_counts[token] = 1

      # Increase count by one
      else:
        word_counts[token] += 1

  return word_counts

<a id="closed"></a>
## 🔒 Creating a Closed Vocabulary

One of the most essential steps in dealing with Textual data is handling Out-of-vocabulary words. This helps the model to handle words which are not present in the training corpus. First step in this process is to create a `closed_vocabulary`. This function creates a closed vocabulary containing only those words according to the `count_threshold` parameter.

In [6]:
def handling_oov(tokenized_sentences, count_threshold) -> 'list':

  # Empty list for closed vocabulary
  closed_vocabulary = []

  # Obtain frequency dictionary using previously defined function
  words_count = count_the_words(tokenized_sentences)

  # Iterate over words and counts
  for word, count in words_count.items():

    # Append if it's more(or equal) to the threshold
    if count >= count_threshold :
      closed_vocabulary.append(word)

  return closed_vocabulary

<a id="unk"></a>
## 🤷🏻 Adding UNK Tokens

In this function we'll add `<unk>` tokens, to those words which are not in the `closed_vocabulary` which we just made.

In [7]:
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>") -> 'list':

  # Convert Vocabulary into a set
  vocabulary = set(vocabulary)

  # Create empty list for sentences
  new_tokenized_sentences = []

  # Iterate over sentences
  for sentence in tokenized_sentences:

    # Iterate over sentence and add <unk>
    # if the token is absent from the vocabulary
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)

    # Append sentece to the new list
    new_tokenized_sentences.append(new_sentence)

  return new_tokenized_sentences

<a id="final"></a>
## 🧼 Final Cleaning Pipeline

In [8]:
def cleansing(train_data, test_data, count_threshold):

  # Get closed Vocabulary
  vocabulary = handling_oov(train_data, count_threshold)

  # Updated Training Dataset
  new_train_data = unk_tokenize(train_data, vocabulary)

  # Updated Test Dataset
  new_test_data = unk_tokenize(test_data, vocabulary)

  return new_train_data, new_test_data, vocabulary

In [9]:
min_freq = 6
final_train, final_test, vocabulary = cleansing(train, test, min_freq)

<a id="build"></a>
# 💪🏻 Building The "Model"

This is a helper function, which will come in handy during inference. This function returns a mapping from n-grams to their frequency in the dataset.

In [10]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>") -> 'dict':

  # Empty dict for n-grams
  n_grams = {}

  # Iterate over all sentences in the dataset
  for sentence in data:

    # Append n start tokens and a single end token to the sentence
    sentence = [start_token]*n + sentence + [end_token]

    # Convert the sentence into a tuple
    sentence = tuple(sentence)

    # Temp var to store length from start of n-gram to end
    m = len(sentence) if n==1 else len(sentence)-1

    # Iterate over this length
    for i in range(m):

      # Get the n-gram
      n_gram = sentence[i:i+n]

      # Add the count of n-gram as value to our dictionary
      # IF n-gram is already present
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      # Add n-gram count
      else:
        n_grams[n_gram] = 1

  return n_grams

This function calculates the priority for the next word given the prior n-gram. This function also implements k-smoothing which helps account for unseen n-grams. Using the previously defined formula:


$$\large
P(w_n|w_{n−N+1:n−1}) = \frac{C(w_{n−N+1:n−1}w_n)}{C(w_{n−N+1:n−1})}
$$

---

### K-smoothing

But what if we come across a n-gram that wasn't in the training set. Then our denominator would would become zero and our definition of probability will become invalid. Thus, we use k-smoothing, which adds a positive constant $k$ to each numerator and $k \times |V|$ in the denominator, where $|V|$ is the number of words in the vocabulary. This ensures any n-gram with zero count has the same probability of $\frac{1}{|V|}$. Thus, our original estimation get's modified to:

$$\large
P(w_n|w_{n−N+1:n−1}) = \frac{C(w_{n−N+1:n−1}w_n) + k}{C(w_{n−N+1:n−1} + k |V|)}
$$

In [11]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k = 1.0) -> 'float':

  # Convert the previous_n_gram into a tuple
  previous_n_gram = tuple(previous_n_gram)

  # Calculating the count, if exists from our freq dictionary otherwise zero
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

  # The Denominator
  denom = previous_n_gram_count + k * vocabulary_size

  # previous n-gram plus the current word as a tuple
  nplus1_gram = previous_n_gram + (word,)

  # Calculating the nplus1 count, if exists from our freq dictionary otherwise zero
  nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  # Numerator
  num = nplus1_gram_count + k

  # Final Fraction
  prob = num / denom
  return prob

Now, we loop over all the words in the vocabulary and then compute their probabilites using our `prob_for_single_word()` fn.

In [12]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0) -> 'dict':

  # Convert to Tuple
  previous_n_gram = tuple(previous_n_gram)

  # Add end and unknown tokens to the vocabulary
  vocabulary = vocabulary + ["<e>", "<unk>"]

  # Calculate the size of the vocabulary
  vocabulary_size = len(vocabulary)

  # Empty dict for probabilites
  probabilities = {}

  # Iterate over words
  for word in vocabulary:

    # Calculate probability
    probability = prob_for_single_word(word, previous_n_gram,
                                           n_gram_counts, nplus1_gram_counts,
                                           vocabulary_size, k=k)
    # Create mapping: word -> probability
    probabilities[word] = probability

  return probabilities

<a id="auto-complete"></a>
# 💬 The Auto-Complete System

Finally, we build our `auto_complete` fn. We simply loop over all the words in the vocabulary assuming that they can be the next word and then return the word with it's probability.

In [13]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0, start_with=None):


    # length of previous words
    n = len(list(n_gram_counts.keys())[0])

    # most recent 'n' words
    previous_n_gram = previous_tokens[-n:]

    # Calculate probabilty for all words
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    # Intialize the suggestion and max probability
    suggestion = None
    max_prob = 0

    # Iterate over all words and probabilites, returning the max.
    # We also add a check if the start_with parameter is provided
    for word, prob in probabilities.items():

        if start_with != None:

            if not word.startswith(start_with):
                continue

        if prob > max_prob:

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

We can also loop over all the various n-gram models to get multiple suggestions. This function just extends from the previously defined function by **taking multiple n-gram counts** instead of one. This allows us to take unigram, bigram, .. counts into account as well.

In [14]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):

    # See how many models we have
    count = len(n_gram_counts_list)

    # Empty list for suggestions
    suggestions = []

    # IMP: Earlier "-1"

    # Loop over counts
    for i in range(count-1):

        # get n and nplus1 counts
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]

        # get suggestions
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        # Append to list
        suggestions.append(suggestion)

    return suggestions

<a id="inference"></a>
# 😊 Inference

Here, we create a list of n-gram counts for a arbitrary range `(1,6)`

In [15]:
n_gram_counts_list = []
for n in range(1, 6):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

Let's give it a sample input of "i was about" in a tokenized manner and get multiple suggestions using the above calculated n-gram counts with smoothing-factor, `k` = 1.0

In [ ]:
previous_tokens = ["i", "was", "about"]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)

In [ ]:
previous_tokens = ["i", "am", "learning"]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)

[Source](https://www.kaggle.com/code/sauravmaheshkar/auto-completion-using-n-gram-models) for this tutorial.